In [159]:
from getpass import getpass
from mysql.connector import connect, Error
import pandas as pd
import numpy as np
import datetime
import re
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import PatternFill
from copy import copy




In [68]:
# получить бизнес-показатели за 3 года по месяцам, пол года по неделям, по дням за 2 месяца

try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        query_months = """WITH new_clients AS (SELECT voo_operations.client_id, MIN(voo_operations.operation_date) AS min_date, "new_client" FROM voo_operations
GROUP BY 1)
SELECT YEAR(voo_operations.operation_date) AS Год, MONTH(voo_operations.operation_date) AS Месяц,
-- SELECT CONCAT(CONCAT(YEAR(voo_operations.operation_date), "_"), MONTH(voo_operations.operation_date)) AS Год_месяц,
ROUND(COUNT(voo_operations.operation_id) / COUNT(DISTINCT voo_operations.department)) AS Операций_на_отделение,
ROUND(COUNT(voo_operations.client_id) / COUNT(DISTINCT voo_operations.client_id), 2) AS Операций_на_клиента,
SUM(IF(departments.city = "Москва", 1, 0)) AS Количество_операций_МСК,
SUM(IF(departments.city = "Санкт-Петербург", 1, 0)) AS Количество_операций_СПБ,
COUNT(voo_operations.operation_id) AS Количество_операций,
ROUND((COUNT(voo_operations.operation_id) / COUNT(DISTINCT voo_operations.department)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_на_отделение,
ROUND(SUM(IF(departments.city = "Москва", 1, 0)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_МСК,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", 1, 0)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_СПБ,
ROUND(COUNT(voo_operations.operation_id) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_всего,
ROUND((SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Приход_на_отделение_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_МСК_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_СПБ_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_всего_млн_$,
ROUND((SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Расход_на_отделение_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_МСК_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_СПБ_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_всего_млн_$,
ROUND((SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Приход_на_отделение_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_МСК_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_СПБ_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_всего_млн_€,
ROUND((SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Расход_на_отделение_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_МСК_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_СПБ_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / selling_rate, 0)) / 1000000, 2) AS Расход_всего_млн_€,
ROUND((SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Оборот_в_валюте_на_отделение_млн,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) / 1000000) AS Оборот_в_валюте_МСК_млн,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) / 1000000) AS Оборот_в_валюте_СПБ_млн,
ROUND(SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / 1000000) AS Оборот_в_валюте_млн,
ROUND((SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000) AS Оборот_авторизованные_на_отделение_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL AND departments.city = "Москва", voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_МСК_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL AND departments.city = "Санкт-Петербург", voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_СПБ_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_млн_руб,
ROUND(SUM(voo_operations.operation_revenue) / COUNT(voo_operations.operation_id)) AS Доход_на_операцию_руб,
ROUND(SUM(voo_operations.operation_revenue) / (SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)))), 2) AS Доход_на_ед_валюты,
ROUND((SUM(voo_operations.operation_revenue) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Доход_на_отделение_млн_руб,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_revenue, 0)) / 1000000, 2) AS Доход_МСК_млн_руб,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_revenue, 0)) / 1000000, 2) AS Доход_СПБ_млн_руб,
ROUND(SUM(voo_operations.operation_revenue)/ 1000000, 2) AS Доход_млн_руб,
ROUND(COUNT(DISTINCT voo_operations.client_id) / COUNT(DISTINCT voo_operations.department)) AS Уникальных_клиентов_на_отделение,
COUNT(DISTINCT IF(departments.city = "Москва", voo_operations.client_id, NULL)) AS Уникальных_клиентов_МСК,
COUNT(DISTINCT IF(departments.city = "Санкт-Петербург", voo_operations.client_id, NULL)) AS Уникальных_клиентов_СПБ,
COUNT(DISTINCT IF(IFNULL(new_clients.new_client, 1) = 1, NULL, voo_operations.client_id)) AS Новых_клиентов,
COUNT(DISTINCT voo_operations.client_id) AS Уникальных_клиентов,
ROUND(SUM(IF(departments.city = "Москва" AND voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT IF(departments.city = "Москва" AND voo_operations.client_id IS NOT NULL, voo_operations.client_id, NULL)))  AS Средний_чек_на_одного_клиента_МСК_руб,
ROUND(SUM(IF(departments.city = "Санкт-Петербург" AND voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT IF(departments.city = "Санкт-Петербург" AND voo_operations.client_id IS NOT NULL, voo_operations.client_id, NULL)))  AS Средний_чек_на_одного_клиента_СПБ_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT voo_operations.client_id))  AS Средний_чек_на_одного_клиента_руб,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) /  COUNT(IF(departments.city = "Москва", voo_operations.operation_id, NULL)))  AS Средний_чек_на_операцию_МСК_уе,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) /  COUNT(IF(departments.city = "Санкт-Петербург", voo_operations.operation_id, NULL)))  AS Средний_чек_на_операцию_СПБ_уе,
ROUND(SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / COUNT(voo_operations.operation_id))  AS Средний_чек_на_операцию_уе

FROM voo_operations 
JOIN departments
ON voo_operations.department = departments.department
LEFT JOIN new_clients 
ON voo_operations.client_id = new_clients.client_id AND voo_operations.operation_date = new_clients.min_date
WHERE voo_operations.operation_date >= CAST(DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 2 YEAR) ,'%Y-01-01') as DATE)
GROUP BY 1, 2
"""
        query_weeks = """WITH new_clients AS (SELECT voo_operations.client_id, MIN(voo_operations.operation_date) AS min_date, "new_client" FROM voo_operations
GROUP BY 1)
SELECT YEAR(voo_operations.operation_date) AS Год, WEEK(voo_operations.operation_date, 1) AS Неделя,
-- SELECT CONCAT(CONCAT(YEAR(voo_operations.operation_date), "_"), MONTH(voo_operations.operation_date)) AS Год_месяц,
ROUND(COUNT(voo_operations.operation_id) / COUNT(DISTINCT voo_operations.department)) AS Операций_на_отделение,
ROUND(COUNT(voo_operations.client_id) / COUNT(DISTINCT voo_operations.client_id), 2) AS Операций_на_клиента,
SUM(IF(departments.city = "Москва", 1, 0)) AS Количество_операций_МСК,
SUM(IF(departments.city = "Санкт-Петербург", 1, 0)) AS Количество_операций_СПБ,
COUNT(voo_operations.operation_id) AS Количество_операций,
ROUND((COUNT(voo_operations.operation_id) / COUNT(DISTINCT voo_operations.department)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_на_отделение,
ROUND(SUM(IF(departments.city = "Москва", 1, 0)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_МСК,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", 1, 0)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_СПБ,
ROUND(COUNT(voo_operations.operation_id) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_всего,
ROUND((SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Приход_на_отделение_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_МСК_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_СПБ_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_всего_млн_$,
ROUND((SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Расход_на_отделение_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_МСК_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_СПБ_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_всего_млн_$,
ROUND((SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Приход_на_отделение_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_МСК_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_СПБ_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_всего_млн_€,
ROUND((SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Расход_на_отделение_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_МСК_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_СПБ_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / selling_rate, 0)) / 1000000, 2) AS Расход_всего_млн_€,
ROUND((SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Оборот_в_валюте_на_отделение_млн,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) / 1000000) AS Оборот_в_валюте_МСК_млн,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) / 1000000) AS Оборот_в_валюте_СПБ_млн,
ROUND(SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / 1000000) AS Оборот_в_валюте_млн,
ROUND((SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000) AS Оборот_авторизованные_на_отделение_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL AND departments.city = "Москва", voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_МСК_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL AND departments.city = "Санкт-Петербург", voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_СПБ_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_млн_руб,
ROUND(SUM(voo_operations.operation_revenue) / COUNT(voo_operations.operation_id)) AS Доход_на_операцию_руб,
ROUND(SUM(voo_operations.operation_revenue) / (SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)))), 2) AS Доход_на_ед_валюты,
ROUND((SUM(voo_operations.operation_revenue) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Доход_на_отделение_млн_руб,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_revenue, 0)) / 1000000, 2) AS Доход_МСК_млн_руб,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_revenue, 0)) / 1000000, 2) AS Доход_СПБ_млн_руб,
ROUND(SUM(voo_operations.operation_revenue)/ 1000000, 2) AS Доход_млн_руб,
ROUND(COUNT(DISTINCT voo_operations.client_id) / COUNT(DISTINCT voo_operations.department)) AS Уникальных_клиентов_на_отделение,
COUNT(DISTINCT IF(departments.city = "Москва", voo_operations.client_id, NULL)) AS Уникальных_клиентов_МСК,
COUNT(DISTINCT IF(departments.city = "Санкт-Петербург", voo_operations.client_id, NULL)) AS Уникальных_клиентов_СПБ,
COUNT(DISTINCT IF(IFNULL(new_clients.new_client, 1) = 1, NULL, voo_operations.client_id)) AS Новых_клиентов,
COUNT(DISTINCT voo_operations.client_id) AS Уникальных_клиентов,
ROUND(SUM(IF(departments.city = "Москва" AND voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT IF(departments.city = "Москва" AND voo_operations.client_id IS NOT NULL, voo_operations.client_id, NULL)))  AS Средний_чек_на_одного_клиента_МСК_руб,
ROUND(SUM(IF(departments.city = "Санкт-Петербург" AND voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT IF(departments.city = "Санкт-Петербург" AND voo_operations.client_id IS NOT NULL, voo_operations.client_id, NULL)))  AS Средний_чек_на_одного_клиента_СПБ_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT voo_operations.client_id))  AS Средний_чек_на_одного_клиента_руб,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) /  COUNT(IF(departments.city = "Москва", voo_operations.operation_id, NULL)))  AS Средний_чек_на_операцию_МСК_уе,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) /  COUNT(IF(departments.city = "Санкт-Петербург", voo_operations.operation_id, NULL)))  AS Средний_чек_на_операцию_СПБ_уе,
ROUND(SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / COUNT(voo_operations.operation_id))  AS Средний_чек_на_операцию_уе

FROM voo_operations 
JOIN departments
ON voo_operations.department = departments.department
LEFT JOIN new_clients 
ON voo_operations.client_id = new_clients.client_id AND voo_operations.operation_date = new_clients.min_date
WHERE voo_operations.operation_date >= CAST(DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 182 DAY) ,'%Y-%m-%d') as DATE)
GROUP BY 1, 2"""
        query_days = """WITH new_clients AS (SELECT voo_operations.client_id, MIN(voo_operations.operation_date) AS min_date, "new_client" FROM voo_operations
GROUP BY 1)
SELECT YEAR(voo_operations.operation_date) AS Год, voo_operations.operation_date AS Дата,
-- SELECT CONCAT(CONCAT(YEAR(voo_operations.operation_date), "_"), MONTH(voo_operations.operation_date)) AS Год_месяц,
ROUND(COUNT(voo_operations.operation_id) / COUNT(DISTINCT voo_operations.department)) AS Операций_на_отделение,
ROUND(COUNT(voo_operations.client_id) / COUNT(DISTINCT voo_operations.client_id), 2) AS Операций_на_клиента,
SUM(IF(departments.city = "Москва", 1, 0)) AS Количество_операций_МСК,
SUM(IF(departments.city = "Санкт-Петербург", 1, 0)) AS Количество_операций_СПБ,
COUNT(voo_operations.operation_id) AS Количество_операций,
ROUND((COUNT(voo_operations.operation_id) / COUNT(DISTINCT voo_operations.department)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_на_отделение,
ROUND(SUM(IF(departments.city = "Москва", 1, 0)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_МСК,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", 1, 0)) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_СПБ,
ROUND(COUNT(voo_operations.operation_id) / (DATEDIFF(MAX(voo_operations.operation_date), MIN(voo_operations.operation_date)) + 1)) AS Сделок_в_день_всего,
ROUND((SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Приход_на_отделение_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_МСК_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_СПБ_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_всего_млн_$,
ROUND((SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Расход_на_отделение_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_МСК_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_СПБ_млн_$,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "USD", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_всего_млн_$,
ROUND((SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Приход_на_отделение_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_МСК_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_СПБ_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Покупка" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.purchase_rate, 0)) / 1000000, 2) AS Приход_всего_млн_€,
ROUND((SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Расход_на_отделение_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR" AND departments.city = "Москва", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_МСК_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR" AND departments.city = "Санкт-Петербург", voo_operations.operation_sum / voo_operations.selling_rate, 0)) / 1000000, 2) AS Расход_СПБ_млн_€,
ROUND(SUM(IF(voo_operations.operation_type = "Продажа" AND voo_operations.currency_type = "EUR", voo_operations.operation_sum / selling_rate, 0)) / 1000000, 2) AS Расход_всего_млн_€,
ROUND((SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Оборот_в_валюте_на_отделение_млн,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) / 1000000) AS Оборот_в_валюте_МСК_млн,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) / 1000000) AS Оборот_в_валюте_СПБ_млн,
ROUND(SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / 1000000) AS Оборот_в_валюте_млн,
ROUND((SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT voo_operations.department)) / 1000000) AS Оборот_авторизованные_на_отделение_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL AND departments.city = "Москва", voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_МСК_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL AND departments.city = "Санкт-Петербург", voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_СПБ_млн_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / 1000000) AS Оборот_авторизованные_млн_руб,
ROUND(SUM(voo_operations.operation_revenue) / COUNT(voo_operations.operation_id)) AS Доход_на_операцию_руб,
ROUND(SUM(voo_operations.operation_revenue) / (SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)))), 2) AS Доход_на_ед_валюты,
ROUND((SUM(voo_operations.operation_revenue) / COUNT(DISTINCT voo_operations.department)) / 1000000, 2) AS Доход_на_отделение_млн_руб,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_revenue, 0)) / 1000000, 2) AS Доход_МСК_млн_руб,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_revenue, 0)) / 1000000, 2) AS Доход_СПБ_млн_руб,
ROUND(SUM(voo_operations.operation_revenue)/ 1000000, 2) AS Доход_млн_руб,
ROUND(COUNT(DISTINCT voo_operations.client_id) / COUNT(DISTINCT voo_operations.department)) AS Уникальных_клиентов_на_отделение,
COUNT(DISTINCT IF(departments.city = "Москва", voo_operations.client_id, NULL)) AS Уникальных_клиентов_МСК,
COUNT(DISTINCT IF(departments.city = "Санкт-Петербург", voo_operations.client_id, NULL)) AS Уникальных_клиентов_СПБ,
COUNT(DISTINCT IF(IFNULL(new_clients.new_client, 1) = 1, NULL, voo_operations.client_id)) AS Новых_клиентов,
COUNT(DISTINCT voo_operations.client_id) AS Уникальных_клиентов,
ROUND(SUM(IF(departments.city = "Москва" AND voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT IF(departments.city = "Москва" AND voo_operations.client_id IS NOT NULL, voo_operations.client_id, NULL)))  AS Средний_чек_на_одного_клиента_МСК_руб,
ROUND(SUM(IF(departments.city = "Санкт-Петербург" AND voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT IF(departments.city = "Санкт-Петербург" AND voo_operations.client_id IS NOT NULL, voo_operations.client_id, NULL)))  AS Средний_чек_на_одного_клиента_СПБ_руб,
ROUND(SUM(IF(voo_operations.client_id IS NOT NULL, voo_operations.operation_sum, 0)) / COUNT(DISTINCT voo_operations.client_id))  AS Средний_чек_на_одного_клиента_руб,
ROUND(SUM(IF(departments.city = "Москва", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) /  COUNT(IF(departments.city = "Москва", voo_operations.operation_id, NULL)))  AS Средний_чек_на_операцию_МСК_уе,
ROUND(SUM(IF(departments.city = "Санкт-Петербург", voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0)), 0)) /  COUNT(IF(departments.city = "Санкт-Петербург", voo_operations.operation_id, NULL)))  AS Средний_чек_на_операцию_СПБ_уе,
ROUND(SUM(voo_operations.operation_sum / (IFNULL(voo_operations.purchase_rate, 0) + IFNULL(voo_operations.selling_rate, 0))) / COUNT(voo_operations.operation_id))  AS Средний_чек_на_операцию_уе

FROM voo_operations 
JOIN departments
ON voo_operations.department = departments.department
LEFT JOIN new_clients 
ON voo_operations.client_id = new_clients.client_id AND voo_operations.operation_date = new_clients.min_date
WHERE voo_operations.operation_date >= CAST(DATE_FORMAT(DATE_SUB(CURDATE(), INTERVAL 60 DAY) ,'%Y-%m-%d') as DATE)
GROUP BY 1, 2"""
        df_months = pd.read_sql(query_months, connection)
        time.sleep(1)
        df_weeks = pd.read_sql(query_weeks, connection)
        time.sleep(1)
        df_days = pd.read_sql(query_days, connection)
except Error as e:
    print(e)

In [169]:
end_file = pd.ExcelWriter(r'D:\FTP-DATA\Analytics\Business_indicators\Данные.xlsx')

df_months.T.to_excel(end_file, sheet_name='Год_месяц', header=False)   
df_weeks.T.to_excel(end_file, sheet_name='Год_неделя', header=False)
df_days.T.to_excel(end_file, sheet_name='Год_день', header=False)

end_file.save()

In [170]:
wb = openpyxl.load_workbook(r'D:\FTP-DATA\Analytics\Business_indicators\Данные.xlsx')

sheets = wb.sheetnames


def as_text(value):
    if value is None:
        return ""
    return str(value)


def make_report(sheet) -> None:
    sheet.insert_cols(1) # вставить пустой столбец в начало
    merged_cells = ['A1:A2', 'A3:A7', 'A8:A11', 'A12:A15', 'A16:A19', 'A20:A23', 'A24:A27', 'A28:A31', 'A32:A35', 'A36:A41', 
                   'A42:A46', 'A47:A52']
    pattern = r'(\d{1,})[^\d]{1,}(\d{1,})'
    
    ranges = [range(int(re.search(pattern, i).group(1)), int(re.search(pattern, i).group(2)) + 1) for i in merged_cells]
    
    dct_ranges = dict(zip(ranges, ['BEFFFE'] + ['C7FFD3', 'FFF4C6'] * 5 + ['C7FFD3']))
    
    new_values = ['Период', 'ВОО_общее', 'Среднее_кол_во_ВОО_в_день', 'Приход_млн_$', 'Расход_млн_$', 'Приход_млн_€', 'Расход_млн_€',
                 'Объем_операций_в_валюте', 'Объем_операций_с_паспортом_млн_руб', 'Доход_млн_руб', 'Кол_во_клиентов',
                 'Средний_чек']
    dct_merge_value = dict(zip(list(map(lambda x: x.split(':')[0], merged_cells)), new_values))
    
    dotted_border = Border(left=Side(style='dotted'), # стиль границ
                     right=Side(style='dotted'), 
                     top=Side(style='dotted'), 
                     bottom=Side(style='dotted'))
    
    for merge_coordinats in merged_cells:  # объединение ячеек и вставка туда значений
        sheet.merge_cells(merge_coordinats)
        sheet[merge_coordinats.split(':')[0]] = dct_merge_value[merge_coordinats.split(':')[0]]
        


    for column_cells in sheet.columns:  # определние и изменение ширины столбцов
        length = max(len(as_text(cell.value)) for cell in column_cells) + 5
        sheet.column_dimensions[get_column_letter(column_cells[0].column)].width = length
    
    max_row_value = sheet.max_row
    max_column_value = sheet.max_column
    
    for row_index in range(1, max_row_value + 1):
        for col_index in range(1, max_column_value + 1):
            
            find_range = list(dct_ranges.keys())[[row_index in key for key in dct_ranges.keys()].index(True)]
            sheet.cell(row_index, col_index).fill = PatternFill(start_color=dct_ranges[find_range], end_color=dct_ranges[find_range], fill_type = "solid") 
            
            sheet.cell(row_index, col_index).border = dotted_border
            if col_index <= 2 or row_index <= 2:
                sheet.cell(row_index, col_index).font = Font(bold=True)
                
            if col_index == 1:
                sheet.cell(row_index, col_index).alignment = Alignment(vertical='center', horizontal='center')
            elif col_index == 2 and row_index >= 3:
                sheet.cell(row_index, col_index).alignment = Alignment(horizontal='left')
            else:
                sheet.cell(row_index, col_index).alignment = Alignment(horizontal='center')
                
            if isinstance(sheet.cell(row_index, col_index).value, (int,)) and row_index >= 3:
                sheet.cell(row_index, col_index).number_format = '# ##0'
            
       
    return None

for sheet in sheets:
    
    make_report(wb[sheet])



    
wb.save(r'D:\FTP-DATA\Analytics\Business_indicators\Данные.xlsx')


In [172]:
wb_source = openpyxl.load_workbook(r'D:\FTP-DATA\Analytics\Business_indicators\Данные.xlsx')

wb_destination = openpyxl.load_workbook(r'D:\FTP-DATA\Analytics\Business_indicators\reserve\Бизнес_показатели.xlsm', read_only=False, keep_vba=True)

sheets = wb_source.sheetnames

for sheet in sheets: # очистка бизнес-показателй
    max_row_value = wb_destination[sheet].max_row
    max_column_value = wb_destination[sheet].max_column
    for row_index in range(1, max_row_value + 1):
        for col_index in range(3, max_column_value + 1):   
            wb_destination[sheet].cell(row_index, col_index).value = None
            wb_destination[sheet].cell(row_index, col_index).fill = PatternFill(fill_type=None, start_color='FFFFFFFF', end_color='FF000000')
            wb_destination[sheet].cell(row_index, col_index).border = Border(left=Side(border_style=None,
                           color='FF000000'),
                 right=Side(border_style=None,
                            color='FF000000'),
                 top=Side(border_style=None,
                          color='FF000000'),
                 bottom=Side(border_style=None,
                             color='FF000000'),
                 diagonal=Side(border_style=None,
                               color='FF000000'),
                 diagonal_direction=0,
                 outline=Side(border_style=None,
                              color='FF000000'),
                 vertical=Side(border_style=None,
                               color='FF000000'),
                 horizontal=Side(border_style=None,
                                color='FF000000')
)
            
for sheet in sheets: # очистка бизнес-показателй
    max_row_value = wb_source[sheet].max_row
    max_column_value = wb_source[sheet].max_column
    for row_index in range(1, max_row_value + 1):
        for col_index in range(3, max_column_value + 1):
            wb_destination[sheet].cell(row_index, col_index).value = wb_source[sheet].cell(row_index, col_index).value
            if wb_source[sheet].cell(row_index, col_index).has_style:
                wb_destination[sheet].cell(row_index, col_index).font = copy( wb_source[sheet].cell(row_index, col_index).font)
                wb_destination[sheet].cell(row_index, col_index).border = copy( wb_source[sheet].cell(row_index, col_index).border)
                wb_destination[sheet].cell(row_index, col_index).fill = copy( wb_source[sheet].cell(row_index, col_index).fill)
                wb_destination[sheet].cell(row_index, col_index).number_format = copy( wb_source[sheet].cell(row_index, col_index).number_format)
                wb_destination[sheet].cell(row_index, col_index).protection = copy( wb_source[sheet].cell(row_index, col_index).protection)
                wb_destination[sheet].cell(row_index, col_index).alignment = copy( wb_source[sheet].cell(row_index, col_index).alignment)
    for column_cells in wb_destination[sheet].columns:  # определние и изменение ширины столбцов
        length = max(len(as_text(cell.value)) for cell in column_cells) + 5
        if get_column_letter(column_cells[0].column).lower() == 'b':
            continue
        else:    
            wb_destination[sheet].column_dimensions[get_column_letter(column_cells[0].column)].width = length            
                
            
wb_destination.save(r'D:\FTP-DATA\Analytics\Business_indicators\Бизнес_показатели.xlsm')            
    
    

